In [64]:
import pandas as pd
import unidecode as un
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/leo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/leo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [65]:
df = pd.read_csv('IMDB_Dataset.csv')
df_old = df

### Preprocessing
We also need to make sure that the sentiments are only positive or negative (in case of faults in data)

We also need to remove HTML tags, due to the reviews being scraped from IMDB

We start by removing special letters etc. with unicode. This will change é to e á to a etc. 

After this we remove all special characters and make the comments clean. 

Removing stopwords

Lemmatizing the data

Finally we also remove unnecessary spaces

In [66]:
# Remove if sentiment is not positive or negative
mask = df['sentiment'].isin(['positive', 'negative'])
df = df[mask]

# Lablenize sentiment
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Remove html tags
df["review"] = df["review"].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())

# Fix decode and allowed_chars
allowed_chars = " abcdefghijklmnopqrstuvwxyz0123456789"
df["review"] = df["review"].apply(lambda x: un.unidecode(x).lower())
df["review"] = df["review"].apply(lambda x: ''.join([i for i in x if i in allowed_chars]))

# Remove stopwords
stop = stopwords.words('english')
df["review"] = df["review"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# Lemmatize
lemmatizer = WordNetLemmatizer()
df["review"] = df["review"].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))


# Strip unnecessary spaces
df["review"] = df["review"].apply(lambda x: x.strip())


/tmp/ipykernel_7560/1069883621.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df["review"] = df["review"].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())


### Split data
The data will be split into training and testing data as well as labels and data

In [67]:
train_size = 40000
train_data, train_labels = df['review'][:train_size], df['sentiment'][:train_size]
test_data, test_labels = df['review'][train_size:], df['sentiment'][train_size:]

### Ngram
Create ngram representation of each word in the review

In [84]:
# ngram
c = CountVectorizer(min_df=0.0, max_df=1.0, binary=False, ngram_range=(1,3))
cv_train_reviews = c.fit_transform(train_data)
cv_test_reviews = c.transform(test_data)

print(f"{cv_train_reviews.shape = }")
print(f"{cv_test_reviews.shape = }")

cv_train_reviews.shape = (40000, 7144506)
cv_test_reviews.shape = (10000, 7144506)
